In [1]:
import os, glob
import numpy as np
import tensorflow as tf
from PIL import Image

from sklearn.model_selection import train_test_split

from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, Input, BatchNormalization, Concatenate, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from tensorflow.keras.applications.resnet50 import ResNet50

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img, array_to_img

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# seed 고정
seed = 42

In [4]:
folder_list = os.listdir('./faces/여자')

In [5]:
# # 이미지 증강용
# def IDG(fname): #파일이름을 변수로 받는 ImageDataGenerator함수
#    ImageDG = ImageDataGenerator(
#        rescale = 1. / 255, # 이미지 value 정규화화
#        rotation_range = 90, # 무작위 회전의 각도 범위
#        brightness_range = [0.3,1.0], # 밝기 범위위
#        fill_mode='nearest' # 인풋 경계의 바깥공간 채우는 방식
#        )
   
#    img = tf.keras.preprocessing.image.load_img(fname) # 이미지파일로 변환
#    x = img_to_array(img)
#    x = x.reshape((1,) + x.shape)
#    i=1
#    save = fname.split('/')[0] + "/" + fname.split('/')[1] + "/" + fname.split('/')[2] + "/" + fname.split('/')[3]
#    # 생성된 파일 저장경로
#    for batch in ImageDG.flow(x, batch_size=1, save_to_dir = save, save_prefix='new'+fname.split('/')[3], save_format='png'):
#        i+=1
#        if i>=10: # 새로 생성되는 데이터 개수 정해주기
#            break

In [6]:
# # 이미지 증강용
# folder_list = os.listdir('./faces/여자')
# fname =  './faces/여자/'
# for f in folder_list:
#     fname =  './faces/여자/' + f +"/"
#     file_list = os.listdir(fname)
#     for i in file_list:
#         filename = fname + i
#         IDG(filename)

In [7]:
# 분류 대상 카테고리 선택하기 
faces_dir = './faces/여자'
woman_categories = [
    '가을','단아','리라','성아','순정','유리','주경','하민'
]
nb_classes = len(woman_categories)
# 이미지 크기 지정 
image_w = 256
image_h = 256
pixels = image_w * image_h * 3
# 이미지 데이터 읽어 들이기 
X = []
Y = []
for idx, cat in enumerate(woman_categories):
    # 레이블 지정 
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    # 이미지 
    image_dir = faces_dir + "/" + cat
    files = glob.glob(image_dir+"/*.png")
    for i, f in enumerate(files):
        img = Image.open(f) 
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)      # numpy 배열로 변환
        X.append(data)
        Y.append(label)
X = np.array(X)
Y = np.array(Y)


In [8]:
print(X.shape)
print(Y.shape)

(5064, 256, 256, 3)
(5064, 8)


In [9]:
# X_std = X/255.0

In [10]:
resnet = ResNet50(weights='imagenet', include_top=False)

In [11]:
for layer in resnet.layers:
    layer.trainable = False

In [12]:
inputs = Input(shape=(256,256,3))
res = resnet(inputs)
gap = GlobalAveragePooling2D()(res)
dense = Dense(128, activation='relu')(gap)
outputs = Dense(nb_classes, activation='softmax')(dense)

In [13]:
model = Model(inputs=inputs, outputs=outputs)

In [14]:
# 모델 확인
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dense_1 (Dense)             (None, 8)                 1032      
                                                                 
Total params: 23,851,016
Trainable params: 263,304
Non-trainable params: 23,587,712
___________________________________________

In [15]:
model.compile(loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [16]:
# 학습 전용 데이터와 테스트 전용 데이터 구분 
X_train, X_test, y_train, y_test = \
    train_test_split(X_std, Y, random_state=seed, test_size=0.2)

In [17]:
import pandas as pd
for i in range(len(pd.DataFrame(y_test).columns)):
    print(pd.DataFrame(y_test)[i].value_counts())

0    870
1    143
Name: 0, dtype: int64
0    883
1    130
Name: 1, dtype: int64
0    880
1    133
Name: 2, dtype: int64
0    898
1    115
Name: 3, dtype: int64
0    896
1    117
Name: 4, dtype: int64
0    898
1    115
Name: 5, dtype: int64
0    872
1    141
Name: 6, dtype: int64
0    894
1    119
Name: 7, dtype: int64


In [18]:
# checkpoint_filepath = "WFMbest.hdf5"

# early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1)

# save_best = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
#     save_weights_only=True, mode='auto', save_freq='epoch', options=None)

In [19]:

hdf5_file = "WFMbest.hdf5"
if os.path.exists(hdf5_file):
    # 기존에 학습된 모델 불러오기기
    model.load_weights(hdf5_file)
else:
    # 학습한 모델이 없으면 파일로 저장
    model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))
    model.save_weights(hdf5_file)

Epoch 1/10
127/127 [==============================] - 351s 2s/step - loss: 1.7056 - accuracy: 0.3565 - val_loss: 1.4410 - val_accuracy: 0.3988
Epoch 2/10
127/127 [==============================] - 321s 3s/step - loss: 1.3098 - accuracy: 0.5315 - val_loss: 1.1206 - val_accuracy: 0.6377
Epoch 3/10
127/127 [==============================] - 321s 3s/step - loss: 1.1043 - accuracy: 0.6159 - val_loss: 0.9822 - val_accuracy: 0.6436
Epoch 4/10
127/127 [==============================] - 325s 3s/step - loss: 0.9713 - accuracy: 0.6579 - val_loss: 0.8137 - val_accuracy: 0.7404
Epoch 5/10
127/127 [==============================] - 339s 3s/step - loss: 0.8735 - accuracy: 0.6890 - val_loss: 0.7336 - val_accuracy: 0.7631
Epoch 6/10
127/127 [==============================] - 340s 3s/step - loss: 0.7788 - accuracy: 0.7341 - val_loss: 0.7162 - val_accuracy: 0.7315
Epoch 7/10
127/127 [==============================] - 321s 3s/step - loss: 0.7315 - accuracy: 0.7448 - val_loss: 0.6457 - val_accuracy: 0.7690